# Understanding our galaxy by using Gaia data

Gaia spacecraft has thoroughly observed our galaxy for 2013-2025, and broadened our understanding of our galaxy's structure & dynamics by enormous magnitude. It has recently been retired, after observing 1 billion stars, 14 times every year. 

Today we will explore Gaia data and try to learn about our galaxy and gain insights from the data.

There are four datasets that I will use: 

1. Stellar data in solar neighborhood (5000 stars with distance < 100 pc) obtained from Gaia Data Release 3 main source catalog
2. Stellar data in our galaxy (10,000 stars with distance < 10 kpc) obtained from Gaia Data Release 3 main source catalog
3. Catalog of open clusters
4. Catalog of globular clusters

In [ ]:
#imports
# astropy imports
import astropy.coordinates as coord
from astropy.table import QTable
from astropy.table import Table
import astropy.units as u


# Other necessary imports
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# gala imports
import gala.dynamics as gd
import gala.potential as gp

## Reading the data from voTable using astropy.table.Table

`Table` object allows you to read `.vot` files very efficiently. 

In [ ]:
stars_near= Table.read("gaia_data_100pc-result.vot")
stars_gal= Table.read("gaia_data_10kpc-result.vot")
op_cl= Table.read("open_clusters.vot")
gl_cl= Table.read("glob_clusters.vot")

Now let's see the Tables

In [ ]:
stars_near[:4]

In [ ]:
stars_gal[:4]

### Description of the Gaia stellar data table columns
----------------------------------------------------------
1. SOURCE_ID:
2. ra:
3. dec:
4. parallax:
5. parallax_error:
6. parallax_over_error:
7. pmra:
8. pmdec:
9. radial_velocity:
10. phot_g_mean_mag:
11. bp_rp:
12. has_rvs:
13. teff_gspphot:
14. logg_gspphot:
15. mh_gspphot:
16. teff_gspspec:
17. logg_gspspec:
18. mh_gspspec:

In [ ]:
op_cl[:4]

Which are RA, DEC, proper motions in RA & DEC, parallax and mode of distances to the individual stars in the cluster. We can take dmode as a proxy for distance here.

In [ ]:
gl_cl[:4]

## Using astropy.coordinates

In [ ]:
#Getting distance from Parallax
#It is just inversion with proper units

dist_n= coord.Distance(parallax=stars_near["parallax"])
dist_g= coord.Distance(parallax=stars_gal["parallax"])

In [ ]:
dist_gc= coord.Distance(parallax= gl_cl["plx"])

It gives error because many plx values of gl_cl contains negative or nan values

In [ ]:
glc_d=gl_cl[gl_cl['plx']>0] #subset with plx>0
dist_gcd=coord.Distance(parallax=glc_d['plx'])

In [ ]:
#3d coordinates of the nearby stars
st_n= coord.SkyCoord(ra=stars_near["ra"],
                      dec=stars_near["dec"],
                      distance=dist_n
                      )
#3d coordinates of the galactic stars
st_g= coord.SkyCoord(ra=stars_gal["ra"],
                      dec=stars_gal["dec"],
                      distance=dist_g
                      )
#3d coordinates of the open clusters
opc= coord.SkyCoord(ra=op_cl["RAJ2000"],
                     dec=op_cl["DEJ2000"],
                     distance=op_cl["dmode"]
                     )
#3d coordinates of globular clusters with distance measures
gld= coord.SkyCoord(ra=glc_d["RAJ2000"],
                     dec=glc_d["DEJ2000"],
                     distance=dist_gcd
                     )
#2D coordinates (RA, DEC) of globular clusters
glc= coord.SkyCoord(ra=gl_cl["RAJ2000"],
                     dec=gl_cl["DEJ2000"],
                     )

In [ ]:
st_n

### Conversion to galactic coordinates is super easy!

In [ ]:
st_n.galactic

In [ ]:
st_n_gal= st_n.galactic
st_g_gal= st_g.galactic
opc_gal= opc.galactic
glc_gal= glc.galactic
gld_gal=gld.galactic

In [ ]:
gc= coord.SkyCoord(ra='17h45m37.2s',dec='-28d56m9.6s', distance=8.1*u.kpc, frame='icrs') #coordinate of galactic center from book
sun= coord.SkyCoord(ra=0*u.deg, dec=0*u.deg, distance=0*u.pc, frame='icrs') #coordinate of the observer

In [ ]:
gc_gal= gc.galactic
sun_gal= sun.galactic

In [ ]:
gc_gal

## Visualizing the data in equatorial and galactic coordinates: distributions of stars, open clusters and globular clusters 

### Distribution of nearby stars

In [ ]:
fig, ax =plt.subplots(figsize=(5,3),dpi=300, subplot_kw={'projection': 'aitoff'})

ax.scatter(st_n.ra.wrap_at(180 * u.deg).radian, st_n.dec.radian, s=0.2)
ax.scatter(gc.ra.wrap_at(180 * u.deg).radian,gc.dec.radian, s=5,c='r')
ax.set_xticklabels([])
ax.set_yticklabels([])
ax.grid(True)

In [ ]:
fig, ax =plt.subplots(figsize=(5,3),dpi=300, subplot_kw={'projection': 'aitoff'})

ax.scatter(st_n_gal.l.wrap_at(180 * u.deg).radian, st_n_gal.b.radian, s=0.2)
ax.scatter(gc_gal.l.wrap_at(180 * u.deg).radian,gc_gal.b.radian, s=5,c='r')
ax.set_xticklabels([])
ax.set_yticklabels([])
ax.grid(True)

In [ ]:
st_nx,st_ny,st_nz=st_n_gal.cartesian.x, st_n_gal.cartesian.y, st_n_gal.cartesian.z

fig, ax = plt.subplots(1,3, figsize=(12,3), dpi=300)
ax[0].scatter(st_nx, st_ny, s=0.5)
ax[0].set_xlabel('x')
ax[0].set_ylabel('y')
ax[1].scatter(st_nx, st_nz,s=0.5)
ax[1].set_xlabel('x')
ax[1].set_ylabel('z')
ax[1].set_yticklabels([])
ax[2].scatter(st_ny, st_nz, s=0.5)
ax[2].set_xlabel('y')
ax[2].set_ylabel('z')
ax[2].set_yticklabels([])

### Distribution of stars within 10 kpc

In [ ]:
fig, ax =plt.subplots(figsize=(5,3),dpi=300, subplot_kw={'projection': 'aitoff'})

ax.scatter(st_g.ra.wrap_at(180 * u.deg).radian, st_g.dec.radian, s=0.2)
ax.scatter(gc.ra.wrap_at(180 * u.deg).radian,gc.dec.radian, s=5,c='r')
ax.set_xticklabels([])
ax.set_yticklabels([])
ax.grid(True)

In [ ]:
fig, ax =plt.subplots(figsize=(6,3),dpi=300, subplot_kw={'projection': 'aitoff'})
im=ax.scatter(st_g.ra.wrap_at(180 * u.deg).radian, 
              st_g.dec.radian, 
              s=0.2, 
              c=st_g.distance, 
              cmap='viridis', 
              vmin=st_g.distance.min().value,
              vmax=st_g.distance.max().value
             )
ax.scatter(gc.ra.wrap_at(180 * u.deg).radian,gc.dec.radian, s=5,c='r')
ax.set_xticklabels([])
ax.set_yticklabels([])
ax.grid(True)
cbar=fig.colorbar(im,ax=ax,shrink=0.7)
cbar.ax.set_ylabel("Distance(pc)")

In [ ]:
fig, ax =plt.subplots(figsize=(6,3),dpi=300, subplot_kw={'projection': 'aitoff'})
im=ax.scatter(st_g_gal.l.wrap_at(180 * u.deg).radian, 
           st_g_gal.b.radian, 
           s=0.2,
           c=st_g_gal.distance, 
           cmap='viridis', 
           vmin=st_g_gal.distance.min().value,
           vmax=st_g_gal.distance.max().value
          )
ax.scatter(gc_gal.l.wrap_at(180 * u.deg).radian,gc_gal.b.radian, s=5,c='r')
ax.set_xticklabels([])
ax.set_yticklabels([])
ax.grid(True)
cbar=fig.colorbar(im,ax=ax,shrink=0.7)
cbar.ax.set_ylabel("Distance(pc)")

In [ ]:
st_gx,st_gy,st_gz=st_g_gal.cartesian.x, st_g_gal.cartesian.y, st_g_gal.cartesian.z

fig, ax = plt.subplots(1,3, figsize=(12,3), dpi=300)
ax[0].scatter(st_gx, st_gy, s=0.5)
ax[0].set_xlabel('x')
ax[0].set_ylabel('y')
ax[1].scatter(st_gx, st_gz,s=0.5)
ax[1].set_xlabel('x')
ax[1].set_ylabel('z')
ax[1].set_yticklabels([])
ax[2].scatter(st_gy, st_gz, s=0.5)
ax[2].set_xlabel('y')
ax[2].set_ylabel('z')
ax[2].set_yticklabels([])

### Distribution of open clusters

In [ ]:
fig, ax =plt.subplots(figsize=(5,3),dpi=300, subplot_kw={'projection': 'aitoff'})
ax.scatter(opc.ra.wrap_at(180 * u.deg).radian, opc.dec.radian, s=2,alpha=0.5)
ax.scatter(gc.ra.wrap_at(180 * u.deg).radian,gc.dec.radian, s=5,c='r')
ax.set_xticklabels([])
ax.set_yticklabels([])
ax.grid(True)

In [ ]:
fig, ax =plt.subplots(figsize=(5,3),dpi=300, subplot_kw={'projection': 'aitoff'})
ax.scatter(opc_gal.l.wrap_at(180 * u.deg).radian, opc_gal.b.radian, s=2,alpha=0.5)
ax.scatter(gc_gal.l.wrap_at(180 * u.deg).radian,gc_gal.b.radian, s=5,c='r')
#ax.scatter(gc.ra.wrap_at(180 * u.deg).radian,gc.dec.radian, s=5,c='r')
ax.set_xticklabels([])
ax.set_yticklabels([])
ax.grid(True)

### Distribution of globular clusters

In [ ]:
fig, ax =plt.subplots(figsize=(5,3),dpi=300, subplot_kw={'projection': 'aitoff'})
ax.scatter(gld.ra.wrap_at(180 * u.deg).radian, gld.dec.radian, s=2)
#ax.scatter(gld.ra.wrap_at(180 * u.deg).radian, gld.dec.radian, s=1, alpha=0.5)
ax.scatter(gc.ra.wrap_at(180 * u.deg).radian,gc.dec.radian, s=5,c='r')
ax.set_xticklabels([])
ax.set_yticklabels([])
ax.grid(True)

In [ ]:
fig, ax =plt.subplots(figsize=(5,3),dpi=300, subplot_kw={'projection': 'aitoff'})
im=ax.scatter(gld_gal.l.wrap_at(180 * u.deg).radian, gld_gal.b.radian, s=2,c=gld_gal.distance, 
           cmap='viridis', 
           vmin=gld_gal.distance.min().value,
           vmax=gld_gal.distance.max().value)
#ax.scatter(gld.ra.wrap_at(180 * u.deg).radian, gld.dec.radian, s=1, alpha=0.5)
ax.scatter(gc_gal.l.wrap_at(180 * u.deg).radian,gc_gal.b.radian, s=5,c='r')
ax.set_xticklabels([])
ax.set_yticklabels([])
ax.grid(True)
cbar=fig.colorbar(im,ax=ax,shrink=0.7)
cbar.ax.set_ylabel("Distance(pc)")

In [ ]:
gld_x,gld_y,gld_z=gld_gal.cartesian.x, gld_gal.cartesian.y, gld_gal.cartesian.z

fig, ax = plt.subplots(1,3, figsize=(12,3), dpi=300)
ax[0].scatter(gld_x, gld_y, s=0.5)
ax[0].set_xlabel('x')
ax[0].set_ylabel('y')
ax[1].scatter(gld_x, gld_z,s=0.5)
ax[1].set_xlabel('x')
ax[1].set_ylabel('z')
ax[1].set_yticklabels([])
ax[2].scatter(gld_y, gld_z, s=0.5)
ax[2].set_xlabel('y')
ax[2].set_ylabel('z')
ax[2].set_yticklabels([])

## Inspecting the stars

In [ ]:
stars_near[:4]

In [ ]:
bp_rp_n= stars_near['bp_rp']
MG_n= stars_near['phot_g_mean_mag']-dist_n.distmod

In [ ]:
fig, ax =plt.subplots(figsize=(4,3), dpi=300)
ax.scatter(bp_rp_n, MG_n, s=0.2, alpha=0.5)
ax.set_xlabel("$G_{BP}-G_{RP}$")
ax.set_ylabel("$M_G$")
ax.invert_yaxis()
ax.set_title("HR diagram for 5000 stars within 100 pc")

In [ ]:
stars_gal[:4]

In [ ]:
bp_rp_g= stars_gal['bp_rp']
MG_g= stars_gal['phot_g_mean_mag']-dist_g.distmod

In [ ]:
fig, ax =plt.subplots(figsize=(4,3), dpi=300)
ax.scatter(bp_rp_g, MG_g, s=0.2, alpha=0.5)
ax.set_xlabel("$G_{BP}-G_{RP}$")
ax.set_ylabel("$M_G$")
ax.invert_yaxis()
ax.set_title("HR diagram for 10000 stars within 10 kpc")

The HR diagram above looks weird. One important thing we are forgetting, i.e. extinction by dust

In [ ]:
bp_rp_g_corr=bp_rp_g-stars_gal['ebpminrp_gspphot']
MG_g_corr=MG_g-stars_gal['ag_gspphot']

In [ ]:
fig, ax =plt.subplots(figsize=(4,3), dpi=300)
ax.scatter(bp_rp_g_corr, MG_g_corr, s=0.2, alpha=0.5)
ax.set_xlabel("$G_{BP}-G_{RP}$")
ax.set_ylabel("$M_G$")
ax.invert_yaxis()
ax.set_title("Extinction corrected HR diagram")

### Thin disk, Thick disk and stellar halo, who lives where

## Using velocity information with coordinates

It turns out, we can also pack velocity info inside SkyCoord objects. 

In [ ]:
#3d coordinates & v of the nearby stars
st_n= coord.SkyCoord(ra=stars_near["ra"],
                      dec=stars_near["dec"],
                      distance=dist_n,
                      pm_ra_cosdec=stars_near["pmra"],
                      pm_dec=stars_near["pmdec"],
                      radial_velocity=stars_near["radial_velocity"]
                      )
#3d coordinates & v of the galactic stars
st_g= coord.SkyCoord(ra=stars_gal["ra"],
                      dec=stars_gal["dec"],
                      distance=dist_g,
                      pm_ra_cosdec=stars_gal["pmra"],
                      pm_dec=stars_gal["pmdec"],
                      radial_velocity=stars_gal["radial_velocity"]
                      )
#3d coordinates of the open clusters
#opc= coord.SkyCoord(ra=op_cl["RAJ2000"],
#                     dec=op_cl["DEJ2000"],
#                     distance=op_cl["dmode"],
#                     
#                     )

In [ ]:
st_n

In [ ]:
st_n_gal=st_n.galactic
st_g_gal=st_g.galactic

### Stellar motion in solar neighborhood

In [ ]:
U,V,W = st_n_gal.velocity.d_xyz
fig, ax =plt.subplots(1,3, figsize=(14,3), dpi=300)
ax[0].scatter(V,U,s=0.2)
ax[0].scatter(0,0,s=5,c='k')
ax[0].set_xlabel("V")
ax[0].set_ylabel("U")
ax[1].scatter(U,W,s=0.2)
ax[1].scatter(0,0,s=5,c='k')
ax[1].set_xlabel("U")
ax[1].set_ylabel("W")
ax[2].scatter(V,W,s=0.2)
ax[2].scatter(0,0,s=5,c='k')
ax[2].set_xlabel("V")
ax[2].set_ylabel("W")

In [ ]:
V.max()

In [ ]:
U,V,W = st_g_gal.velocity.d_xyz
fig, ax =plt.subplots(1,3, figsize=(14,3), dpi=300)
ax[0].scatter(V,U,s=0.2)
ax[0].scatter(0,0,s=5,c='k')
ax[0].set_xlabel("V")
ax[0].set_ylabel("U")
ax[1].scatter(U,W,s=0.2)
ax[1].scatter(0,0,s=5,c='k')
ax[1].set_xlabel("U")
ax[1].set_ylabel("W")
ax[2].scatter(V,W,s=0.2)
ax[2].scatter(0,0,s=5,c='k')
ax[2].set_xlabel("V")
ax[2].set_ylabel("W")

In [ ]:
V.max()

In [ ]:
st_n_lsr=st_n.galacticlsr
st_n_lsr

In [ ]:
U,V,W = st_n_lsr.velocity.d_xyz
fig, ax =plt.subplots(1,3, figsize=(14,3), dpi=300)
ax[0].scatter(V,U,s=0.2)
ax[0].scatter(0,0,s=5,c='k')
ax[0].set_xlabel("V")
ax[0].set_ylabel("U")
ax[1].scatter(U,W,s=0.2)
ax[1].scatter(0,0,s=5,c='k')
ax[1].set_xlabel("U")
ax[1].set_ylabel("W")
ax[2].scatter(V,W,s=0.2)
ax[2].scatter(0,0,s=5,c='k')
ax[2].set_xlabel("V")
ax[2].set_ylabel("W")

### apply_space_motion, Watch the stars dance!

In [ ]:
st_n_gal

In [ ]:
def star_plot(star_coords, vmin=None, vmax=None):
    """
    Take a list of stars as SkyCoord objects and build a figure showing the stars projected onto
    the plane of the sky using the Aitoff projection, and in three dimensional space centered
    on the Earth. In both plots the points are colored by distance (from Earth).

    Parameters
    ----------
    star_coords : Astropy SkyCoord object
        The coordinates of the objects to be plotted
    vmin, vmax : float
        Optional min and max values in pc for the colormap used to color the plot by distance.
        If not set they will be the minimum and maximum distances of star_coords.

    Returns
    -------
    response : matplotlib figure

    """

    # Set the min and max values for the colormap if not given
    if not vmin:
        vmin = star_coords.distance.min().value

    if not vmax:
        vmax = star_coords.distance.max().value

    # Initialize the figure
    fig = plt.figure(figsize=(10, 4),dpi=300)

    # Sky Projection plot
    ax1 = fig.add_subplot(1, 2, 1, projection="aitoff")  # Add left subplot

    # Turn off the axis ticks and labels, and turn on the plot grid
    ax1.set_xticklabels([])
    ax1.set_yticklabels([])
    ax1.grid(True)

    # Plot the stars, colored by distance
    ax1.scatter(
        star_coords.ra.wrap_at(180 * u.deg).radian,
        star_coords.dec.radian,
        c=star_coords.distance,
        vmin=vmin,
        vmax=vmax,
        marker="*",
        s=200,
        cmap="plasma",
    )

    # 3D plot
    ax2 = fig.add_subplot(1, 2, 2, projection="3d")  # Add right subplot

    # Turn off axis ticks and labels
    ax2.set_xticklabels([])
    ax2.set_yticklabels([])
    ax2.set_zticklabels([])

    # Plotting the Earth as a black circle
    ax2.scatter([0], [0], [0], s=100, c="k", marker="o")

    # Plot the stars, colored by distance
    pc = ax2.scatter(
        star_coords.cartesian.x,
        star_coords.cartesian.y,
        star_coords.cartesian.z,
        c=star_coords.distance,
        s=200,
        vmin=vmin,
        vmax=vmax,
        marker="*",
        cmap="plasma",
    )

    # Adding the colorbar
    cbar = fig.colorbar(pc, shrink=0.6, location="right")
    cbar.ax.tick_params(labelsize=14)
    cbar.ax.set_ylabel("Distance (pc)", fontsize=18)

    # Remove extra space between the subplots
    fig.subplots_adjust(wspace=0)

    # This prevents getting an extra copy of the plot when you call the function
    plt.close()

    return fig

In [ ]:
import warnings  # So we can suppress expected warnings
import matplotlib.animation as animation

plt.rcParams["animation.html"] = (
    "jshtml"  # To make the animations render correctly in the notebook
)

In [ ]:
def star_animation(star_coords, evolution_time, steps, vmin=None, vmax=None):
    """
    Take a list of stars as a SkyCoord object and build an animation showing how the star's
    positions evolve over time.

    Two subplots are produced, the left showing the stars projected onto the plane of the sky
    using the Aitoff projection, and the right in three dimensional space centered on the Earth.
    In both plots the points are colored by distance (from Earth).

    Parameters
    ----------
    star_coords : Astropy SkyCoord object
        The coordinates of the objects you wish to plot
    evolution_time : float or Astropy Quantity
        The amount of time to evolve the stellar positions each step.
        If not specified as a quantity, years will be assumed.
    steps : int
        The number of time steps to take.
        The total amount of time the system will be eveolved for is evolution_time * steps
    vmin, vmax : float
        Optional min and max values for the colormap used to color the plot by distance.
        If not set they will be the minimum and maximum distances of the input stars at present time.

    Returns
    -------
    response : matplotlib figure

    """

    # Make sure the evolution time is in years
    if not isinstance(evolution_time, u.Quantity):
        evolution_time *= u.yr

    # Create array of time deltas (all in relation to present time)
    dt_array = np.linspace(0, evolution_time * steps, steps, endpoint=False)

    # If vmin/vmax were not set we need to set them (if they are not set at all,
    # each step of the animation will have different colormap boundaries).
    if not vmin:
        vmin = star_coords.distance.min().value

    if not vmax:
        vmax = star_coords.distance.max().value

    def animate(iteration, dt_array, aitoff_scatter, cube_scatter):
        """
        This function handles updating the plot for each frame of the animation.

        Parameters
        ----------
        iteration: int
            Current iteration (frame number)
        dt_array : Quantity
            Array of time deltas
        aitoff_scatter : matplotlib PathCollection
            A matplotlib collection object that holds the 2D (left) plot data points.
        cube_scatter : matplotlib Path3DCollection
            A matplotlib collection object that holds the 3D (right) plot data points.
        """

        with warnings.catch_warnings():
            # Projecting more than 5 years into the future gives a "dubious year" warning
            # due to the unpredictability of leap seconds, we suppress this warning becuase
            # we are not concerned with temporal accuracy to the second
            warnings.filterwarnings("ignore", message="ERFA function ")
            new_pos = star_coords.apply_space_motion(dt=dt_array[iteration])

        # set_offsets sets the point locations to the newly calculated coordinates
        aitoff_scatter.set_offsets(
            list(zip(new_pos.ra.wrap_at(180 * u.deg).radian, new_pos.dec.radian))
        )

        # set_array sets the colors of the points based on their new distances
        aitoff_scatter.set_array(new_pos.distance)

        # For the 3D plot we have to use the private property _offsets3d to update the point locations
        cube_scatter._offsets3d = (
            new_pos.cartesian.x,
            new_pos.cartesian.y,
            new_pos.cartesian.z,
        )

        # set_array sets the colors of the points based on their new distances
        cube_scatter.set_array(new_pos.distance)

        return (
            aitoff_scatter,
            cube_scatter,
        )

    # Call our star_plot function from the previous section to initialize the figure
    fig = star_plot(star_coords, vmin=vmin, vmax=vmax)

    # Get the matplotlib Collection objects that correspond to our sets of points
    aitoff_scatter = fig.axes[0].collections[0]
    cube_scatter = fig.axes[1].collections[1]

    # Build the animation
    ani = animation.FuncAnimation(
        fig,  # The figure we are animating
        animate,  # The function that updates the plot for each frame
        fargs=(dt_array, aitoff_scatter, cube_scatter),  # Args for the animate function
        frames=steps,  # The number of frames in our animation
        interval=100,
    )  # Delay between frames in milliseconds

    # This prevents getting an extra copy of the first frame of the plot when you call the function
    plt.close()

    return ani

In [ ]:
star_plot(st_n[:100])

In [ ]:
star_animation(st_n[:100], 10000*u.yr, 20)

### How fast is the sun rotating? Measuring the Oort's constants

In [ ]:
fig, ax =plt.subplots(dpi=300)
ax.scatter(st_g_gal.l.radian, st_g_gal.radial_velocity,s=2)

In [ ]:
fig, ax =plt.subplots(dpi=300)
ax.scatter(st_g_gal.l.radian, np.sqrt((st_g_gal.pm_b.value)**2+(st_g_gal.pm_l_cosb.value)**2),s=2)

### The LSR frame

In [ ]:
st_n_lsr=st_n.galacticlsr
st_n_lsr

In [ ]:
U,V,W=

### The Galactocentric frame

## Computing galactic orbits with gala

In [ ]:
st_n_galcen= st_n_gal.galactocentric

In [ ]:
st_n_galcen

In [ ]:
#plt.hist(galcen.z.value)

In [ ]:
milky_way = gp.MilkyWayPotential()
milky_way

In [ ]:
H = gp.Hamiltonian(milky_way)

In [ ]:
w0 = gd.PhaseSpacePosition(galcen.cartesian)
w0

In [ ]:
orbits = H.integrate_orbit(w0, dt=1*u.Myr, t1=0*u.Myr, t2=500*u.Myr)

In [ ]:
fig = orbits[:,:100].plot()

In [ ]:
plt.close()